In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kenai,US,2021-03-27 15:17:07,60.5544,-151.2583,14.00,79,75,3.44
1,1,Rikitea,PF,2021-03-27 15:17:07,-23.1203,-134.9692,78.55,73,33,20.09
2,2,Khatanga,RU,2021-03-27 15:17:07,71.9667,102.5000,4.41,94,100,6.93
3,3,Punta Arenas,CL,2021-03-27 15:12:37,-53.1500,-70.9167,57.20,67,75,11.50
4,4,Hobart,AU,2021-03-27 15:13:32,-42.8794,147.3294,53.60,71,75,11.50


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
#heat_layer = gmaps.heatmap_layer(locations, weights=temps)
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Rikitea,PF,2021-03-27 15:17:07,-23.1203,-134.9692,78.55,73,33,20.09
13,13,Neiafu,TO,2021-03-27 15:17:11,-18.6500,-173.9833,78.80,89,20,9.22
29,29,Salalah,OM,2021-03-27 15:17:16,17.0151,54.0924,84.20,54,0,5.75
31,31,Alofi,NU,2021-03-27 15:17:17,-19.0595,-169.9187,78.80,89,14,8.05
32,32,Balaka,MW,2021-03-27 15:17:17,-14.9793,34.9558,75.27,87,44,3.31
34,34,Vaini,TO,2021-03-27 15:17:18,-21.2000,-175.2000,78.80,94,20,5.75
35,35,Carnarvon,AU,2021-03-27 15:15:10,-24.8667,113.6333,78.80,73,1,10.36
37,37,Castro,BR,2021-03-27 15:17:19,-24.7911,-50.0119,81.03,49,88,6.15
38,38,Avarua,CK,2021-03-27 15:17:19,-21.2078,-159.7750,78.80,89,75,5.75
39,39,Dingle,PH,2021-03-27 15:17:19,10.9995,122.6711,77.22,84,23,6.85


In [16]:
preferred_cities_df.notnull().sum()
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID       187
City          187
Country       187
Date          187
Lat           187
Lng           187
Max Temp      187
Humidity      187
Cloudiness    187
Wind Speed    187
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,78.55,-23.1203,-134.9692,
13,Neiafu,TO,78.80,-18.6500,-173.9833,
29,Salalah,OM,84.20,17.0151,54.0924,
31,Alofi,NU,78.80,-19.0595,-169.9187,
32,Balaka,MW,75.27,-14.9793,34.9558,
34,Vaini,TO,78.80,-21.2000,-175.2000,
35,Carnarvon,AU,78.80,-24.8667,113.6333,
37,Castro,BR,81.03,-24.7911,-50.0119,
38,Avarua,CK,78.80,-21.2078,-159.7750,
39,Dingle,PH,77.22,10.9995,122.6711,


In [20]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [27]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,78.55,-23.1203,-134.9692,Pension Maro'i
13,Neiafu,TO,78.80,-18.6500,-173.9833,Mystic Sands
29,Salalah,OM,84.20,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
31,Alofi,NU,78.80,-19.0595,-169.9187,Taloa Heights
32,Balaka,MW,75.27,-14.9793,34.9558,Sun Village Lodge
34,Vaini,TO,78.80,-21.2000,-175.2000,Keleti Beach Resort
35,Carnarvon,AU,78.80,-24.8667,113.6333,Hospitality Carnarvon
37,Castro,BR,81.03,-24.7911,-50.0119,CHACARA BAILLY
38,Avarua,CK,78.80,-21.2078,-159.7750,Paradise Inn
39,Dingle,PH,77.22,10.9995,122.6711,SEGAYA PROPERTY


In [31]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [37]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))